# Setup

In [1]:
import contracts
contracts.disable_all()
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html
dw.logger.setLevel(50)

INFO:dt-world:duckietown-world 5.0.10
INFO:zuper-typing:zuper-typing 5.0.12
INFO:zuper-commons:zuper-commons 5.0.7


Better visualization of output

In [2]:
%%html
<style>
pre {line-height: 90%}
</style>

# Map representation

Let's load a map and see how data is represented inside:

In [3]:
m = dw.load_map('4way')
ipython_draw_html(m);

The map is a `DuckietownMap` which is an instance of `PlacedObject`. All spatially situated objects
are instances of this class.

In [4]:
type(m).mro() # see all superclasses of the object

[duckietown_world.world_duckietown.duckietown_map.DuckietownMap,
 duckietown_world.geo.placed_object.PlacedObject,
 duckietown_serialization_ds1.serialization1.Serializable,
 duckietown_serialization_ds1.serialization1.Serializable0,
 object]

## Children

The data is arranged in a hierarchy. We can visualize using the function `get_object_tree`. The hierarchy is more deep, but we clip it to 2 levels:

In [5]:
print(dw.get_object_tree(m, levels=4))

DuckietownMap

├ ob00-trafficlight ┐ TrafficLight
│                   └───────────────
└ tilemap ┐ TileMap
          │ 
          │ ├ tile-0-4 ┐ Tile
          │ │          │
          │ │          │ └ curve_left ┐ PlacedObject
          │ │          │              │
          │ │          │              │ └ curve ┐ PlacedObject
          │ │          │              │         └───────────────
          │ │          │              └───────────────────────────
          │ │          └────────────────────────────────────────────
          │ ├ tile-1-4 ┐ Tile
          │ │          │
          │ │          │ └ straight ┐ PlacedObject
          │ │          │            │
          │ │          │            │ ├ lane1 ┐ LaneSegment
          │ │          │            │ │       └──────────────
          │ │          │            │ └ lane2 ┐ LaneSegment
          │ │          │            │         └──────────────
          │ │          │            └──────────────────────────
          │ │   

The children are available in the `children` variable:

In [6]:
m.children

{'ob00-trafficlight': TrafficLight(children={}, spatial_relations={}),
 'tilemap': TileMap(children={'tile-0-4': Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_relations={1: GroundTruth(() -> ('curve_left',)  SE2Transform([0.0, 0.0],0.0))}), 'tile-1-4': Tile(children={'straight': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transfor

In [7]:
m.children['tilemap'].children

{'tile-0-4': Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_relations={1: GroundTruth(() -> ('curve_left',)  SE2Transform([0.0, 0.0],0.0))}),
 'tile-1-4': Tile(children={'straight': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],3.141592653589793))})}, spatial_relations={1: GroundTruth(() -> ('straight',)  SE2Tran

You can use the notation below to get a child in a compact way:

In [8]:
lane_segment = m['tilemap/tile-0-1/straight/lane1']

In this case we can see how a Tile has a child `curve_left` with a child `curve` with two children `lane1` and `lane2`:

In [9]:
tile = m.children['tilemap'].children['tile-0-0']
print(dw.get_object_tree(tile, attributes=False, levels=10))

Tile

└ curve_left ┐ PlacedObject
             │ 
             │ └ curve ┐ PlacedObject
             │         │
             │         │ ├ lane1 ┐ LaneSegment
             │         │ │       └──────────────
             │         │ └ lane2 ┐ LaneSegment
             │         │         └──────────────
             │         └──────────────────────────
             └──────────────────────────────────────


In [10]:
print(dw.get_object_tree(tile, attributes=True, levels=10))

Tile

 kind: curve_left
 drivable: true
 

└ curve_left ┐ PlacedObject
             │ 
             │ └ curve ┐ PlacedObject
             │         │
             │         │ ├ lane1 ┐ LaneSegment
             │         │ │       │
             │         │ │       │  width: 0.376
             │         │ │       │  control_points:
             │         │ │       │  -   ~SE2Transform:
             │         │ │       │          p:
             │         │ │       │          - -0.5
             │         │ │       │          - -0.22
             │         │ │       │  -   ~SE2Transform:
             │         │ │       │          theta_deg: -45
             │         │ │       │          p:
             │         │ │       │          - -0.3
             │         │ │       │          - -0.3
             │         │ │       │  -   ~SE2Transform:
             │         │ │       │          theta_deg: -90
             │         │ │       │          p:
             │         │ │       │    

In [11]:
lane1 = tile['curve_left/curve/lane1']
lane2 = tile['curve_left/curve/lane2']
ipython_draw_html(lane1);
ipython_draw_html(lane2);

In [12]:
curve = tile['curve_left/curve']
print(dw.get_object_tree(curve, attributes=True, spatial_relations=True, levels=10))

PlacedObject

├ lane1 ┐ LaneSegment
│       │ 
│       │  width: 0.376
│       │  control_points:
│       │  -   ~SE2Transform:
│       │          p:
│       │          - -0.5
│       │          - -0.22
│       │  -   ~SE2Transform:
│       │          theta_deg: -45
│       │          p:
│       │          - -0.3
│       │          - -0.3
│       │  -   ~SE2Transform:
│       │          theta_deg: -90
│       │          p:
│       │          - -0.22
│       │          - -0.5
│       │ 
│       └──────────────────────────
└ lane2 ┐ LaneSegment
        │ 
        │  width: 0.376
        │  control_points:
        │  -   ~SE2Transform:
        │          p:
        │          - -0.5
        │          - -0.22
        │  -   ~SE2Transform:
        │          theta_deg: 45
        │  -   ~SE2Transform:
        │          theta_deg: 90
        │          p:
        │          - 0.22
        │          - 0.5
        │ 
        └─────────────────────────

- from "." to "lane1"  SE2Transform([0

In [13]:
ipython_draw_html(curve);

In [14]:
lane = tile['curve_left/curve/lane2']._copy()

ipython_draw_html(lane);

In [15]:
lane.width

0.376

In [16]:
lane.control_points

[SE2Transform([-0.5, -0.22],0.0),
 SE2Transform([0.0, 0.0],0.7853981633974483),
 SE2Transform([0.22, 0.5],1.5707963267948966)]

## Parametrization of lanes

The lane is parametrized with a parameter `beta` that interpolates among the control points.

Here we create an animation of the center point for different `beta`s.

In [17]:
import numpy as np
npoints = len(lane.control_points)
betas = list(np.linspace(-1, npoints + 1, 20))
print(betas)

[-1.0, -0.736842105263158, -0.4736842105263158, -0.21052631578947367, 0.05263157894736836, 0.3157894736842104, 0.5789473684210527, 0.8421052631578947, 1.1052631578947367, 1.3684210526315788, 1.6315789473684208, 1.8947368421052628, 2.1578947368421053, 2.4210526315789473, 2.6842105263157894, 2.9473684210526314, 3.2105263157894735, 3.473684210526316, 3.7368421052631575, 4.0]


In [18]:
transforms = []
for beta in betas:
    # call the function `center_point` to get the center point (in SE(2))
    p = lane.center_point(beta)
    transform = dw.SE2Transform.from_SE2(p)
    transforms.append(transform)

ground_truth = dw.SampledSequence[dw.SE2Transform](betas, transforms)
lane.set_object('traveling-point', dw.PlacedObject(), ground_truth=ground_truth)
ipython_draw_html(lane);

Scrub the timeline to see the animation.

In [19]:
import geometry as geo
q = geo.SE2_from_translation_angle([+0.05, -0.05], np.deg2rad(20))

In [20]:
lane.set_object('db18-4', dw.DB18(), ground_truth=dw.SE2Transform.from_SE2(q))
lane_pose = lane.lane_pose_from_SE2(q)

lane.set_object('marker3', dw.PlacedObject(), ground_truth=lane_pose.center_point)

In [21]:
lane.children

{'traveling-point': PlacedObject(children={}, spatial_relations={}),
 'db18-4': DB18(children={}, spatial_relations={}),
 'marker3': PlacedObject(children={}, spatial_relations={})}

In [22]:
lane.spatial_relations

{0: GroundTruth(() -> ('traveling-point',)  SampledSequence[SE2Transform](timestamps=[-1.0, -0.736842105263158, -0.4736842105263158, -0.21052631578947367, 0.05263157894736836, 0.3157894736842104, 0.5789473684210527, 0.8421052631578947, 1.1052631578947367, 1.3684210526315788, 1.6315789473684208, 1.8947368421052628, 2.1578947368421053, 2.4210526315789473, 2.6842105263157894, 2.9473684210526314, 3.2105263157894735, 3.473684210526316, 3.7368421052631575, 4.0], values=[SE2Transform([-0.6, -0.2199999988079071],0.0), SE2Transform([-0.5736842105263158, -0.2199999988079071],0.0), SE2Transform([-0.5473684210526316, -0.2199999988079071],0.0), SE2Transform([-0.5210526315789473, -0.2199999988079071],0.0), SE2Transform([-0.470525719458915, -0.21874729127170855],0.04133674659227066), SE2Transform([-0.32530923670416145, -0.19434484508076877],0.2480204795536241), SE2Transform([-0.18819117546714098, -0.1406611089369041],0.4547042125149777), SE2Transform([-0.0650081458829751, -0.059981201486350694],0.661

In [23]:
ipython_draw_html(lane);

In [24]:
assert lane_pose.correct_direction

In [25]:
assert lane_pose.inside

In [26]:
assert lane